In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
results_dir_relu = "results/classification/single_layer/relu/breastcancer"
results_dir_tanh = "results/classification/single_layer/tanh/breastcancer"
model_names_relu = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]
model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh"]



full_config_path = "breast_cancer_N455_p30"

relu_fits = get_model_fits(
    config=full_config_path,
    results_dir=results_dir_relu,
    models=model_names_relu,
    include_prior=False,
)

tanh_fits = get_model_fits(
    config=full_config_path,
    results_dir=results_dir_tanh,
    models=model_names_tanh,
    include_prior=False,
)


In [3]:
from utils.generate_data import load_breast_cancer_data
X_train, X_test, y_train, y_test, *_ = load_breast_cancer_data(
    test_size=0.2, standardize=False, random_state=42
)

In [4]:
from sklearn.metrics import accuracy_score, log_loss
from scipy.stats import mode
import pandas as pd
import numpy as np

model_names = list(tanh_fits.keys())
results = []

for model in model_names:
    posterior = tanh_fits[model]['posterior']
    
    # Get predicted class labels (shape: [n_samples, n_test])
    pred_samples = posterior.stan_variable("pred_test")
    majority_preds = mode(pred_samples, axis=0, keepdims=False).mode.flatten()

    # Compute accuracy
    acc = accuracy_score(y_test, majority_preds)

    # Get predicted probabilities (shape: [n_samples, n_test, n_classes])
    pred_probs = posterior.stan_variable("prob_test")  # e.g., shape (4000, 114, 2)
    mean_probs = pred_probs.mean(axis=0)  # shape: [n_test, 2]

    # Adjust y_test to 0-based indexing for log_loss
    y_test_adj = y_test - 1
    nll = log_loss(y_test_adj, mean_probs, labels=[0, 1])

    results.append({
        "Model": model,
        "Accuracy": acc,
        "NLL": nll
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)



In [ ]:
# Write LaTeX table
latex_table = results_df.to_latex(index=False, float_format="%.4f", column_format="lcc", caption="Accuracy and NLL per model.", label="tab:accuracy_nll")
print(latex_table)


In [ ]:
print("Of 114 observations,", np.round(114*results_df['Accuracy'][0], 3), "were classified correctly by the", results_df['Model'][0], "model \n")
print("Of 114 observations,", np.round(114*results_df['Accuracy'][1], 3), "were classified correctly by the", results_df['Model'][1], "model \n")
print("Of 114 observations,", np.round(114*results_df['Accuracy'][2], 3), "were classified correctly by the", results_df['Model'][2], "model \n")
print("Of 114 observations,", np.round(114*results_df['Accuracy'][3], 3), "were classified correctly by the", results_df['Model'][3], "model \n")

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

X_test_df = pd.DataFrame(X_test)

W_1 = tanh_fits['Gaussian tanh']['posterior'].stan_variable("W_1")[0, :, :]   # (30, 16)
W_2 = tanh_fits['Gaussian tanh']['posterior'].stan_variable("W_L")[0, :, :]   # (16, 2)
b_1 = tanh_fits['Gaussian tanh']['posterior'].stan_variable("hidden_bias")[0, :]  # (16,)
b_2 = tanh_fits['Gaussian tanh']['posterior'].stan_variable("output_bias")[0, :]  # (2,)

# Konverter DataFrame til tensor
X = torch.tensor(X_test_df.to_numpy(), dtype=torch.float32)  # (114, 30)

# Stan-vekter til tensor
W1 = torch.tensor(W_1, dtype=torch.float32)   # (30, 16)
b1 = torch.tensor(b_1.squeeze(), dtype=torch.float32)  # (16,)
W2 = torch.tensor(W_2, dtype=torch.float32)   # (16, 2)
b2 = torch.tensor(b_2, dtype=torch.float32)   # (2,)

# Definer nettverket
class StanNNClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, activation=torch.tanh):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        self.activation = activation

    def forward(self, x):
        h = self.activation(self.linear1(x))
        logits = self.linear2(h)
        return logits  # matcher Stan sin `output`

    def predict_proba(self, x):
        logits = self.forward(x)
        return F.softmax(logits, dim=1)  # matcher Stan sin `prob_test`

    def predict(self, x):
        probs = self.predict_proba(x)
        return torch.argmax(probs, dim=1)  # matcher Stan sin `pred_test`

# Bygg og kopier vektene
model = StanNNClassifier(input_dim=30, hidden_dim=16, output_dim=2, activation=torch.tanh)
with torch.no_grad():
    model.linear1.weight.copy_(W1.T)   # (16, 30)
    model.linear1.bias.copy_(b1)       # (16,)
    model.linear2.weight.copy_(W2.T)   # (2, 16)
    model.linear2.bias.copy_(b2)       # (2,)

# === Test forward ===
logits = model(X)                # (114, 2), Stan: `output_test`
probs = model.predict_proba(X)   # (114, 2), Stan: `prob_test`
preds = model.predict(X)         # (114,),   Stan: `pred_test`

#print("logits shape:", logits.shape)
#print("probs shape :", probs)
#print("preds shape :", preds)


In [ ]:
stan_probs = tanh_fits['Gaussian tanh']['posterior'].stan_variable("prob_test")[0, :, :]
stan_probs_torch = torch.tensor(stan_probs, dtype=torch.float32)

print("Max diff:", (probs - stan_probs_torch).abs().max().item())


In [ ]:
from collections import defaultdict
import pandas as pd
from utils.robust_utils import estimate_robustness_over_test_set
import torch
# === Settings ===
epsilons = [0.01, 0.05, 0.1, 0.25]
#epsilons = [0.1, 0.5, 1.0, 2.5]
scales = [0.01, 0.05, 0.1, 0.5, 1.0, 5.0]
sample_indices = range(0, 4000, 800)
input_dim = X_test.shape[1]
hidden_dim = 16
output_dim = 2
p_norm = 2

# === Subset test set ===
X_test_df = pd.DataFrame(X_test)
y_test_s = pd.Series(y_test) - 1  # Ensure labels in {0,1}
test_subset = X_test_df.sample(frac=1.0, random_state=42)
subset_indices = test_subset.index
y_subset = y_test_s.loc[subset_indices]

# === Run robustness for each model ===
all_results = []

for model_name, fit_entry in relu_fits.items():
    for epsilon in epsilons:
        for scale in scales:
            delta = scale * epsilon

            df_result = estimate_robustness_over_test_set(
                x_test=test_subset,
                y_test=y_subset,
                fits_dict={model_name: fit_entry},  # Wrap as dict
                input_dim=input_dim,
                hidden_dim=hidden_dim,
                output_dim=output_dim,
                sample_indices=sample_indices,
                epsilon=epsilon,
                delta=delta,
                p_norm=p_norm,
                activation=F.relu
            )

            df_result["epsilon"] = epsilon
            df_result["delta"] = delta
            df_result["scale"] = round(scale, 2)
            df_result["model"] = model_name
            all_results.append(df_result.copy())

# === Combine results ===
df_robust = pd.concat(all_results, ignore_index=True)

# === Unpack 'robustness' dictionary column into separate columns ===
df_flat = pd.concat(
    [df_robust.drop(columns=["robustness"]),
     df_robust["robustness"].apply(pd.Series)],
    axis=1
)

# === Add derived columns ===
df_flat["1-p1"] = (1.0 - df_flat["p1"]).round(5)
df_flat["1-p2"] = (1.0 - df_flat["p2"]).round(5)

# Resulting DataFrame: df_flat


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Ensure model name consistency and ordering
#model_order = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh"]
model_order = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]
#Ns = sorted(df_flat["N"].unique())

# Global color scale (consistent across all plots)
vmin = df_flat["1-p1"].min()
vmax = df_flat["1-p1"].max()

short_names = {
    "Gaussian": "Gauss",
    "Regularized Horseshoe": "RHS",
    "Dirichlet Horseshoe": "DHS",
    "Dirichlet Student T": "DS-T",
}

# short_names = {
#     "Gaussian tanh": "Gauss",
#     "Regularized Horseshoe tanh": "RHS",
#     "Dirichlet Horseshoe tanh": "DHS",
#     "Dirichlet Student T tanh": "DS-T",
# }

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8), sharex=True, sharey=True)

for j, model in enumerate(model_order):
    row, col = divmod(j, 2)
    ax = axes[row, col]
    df_model = df_flat[df_flat["model"] == model]

    # Use pivot_table to handle duplicates
    heatmap_df = df_model.pivot_table(
        index="scale", columns="epsilon", values="1-p1", aggfunc="mean"
    )

    sns.heatmap(
        heatmap_df.sort_index(ascending=False),
        annot=True, fmt=".2f", cmap="RdYlGn", ax=ax,
        cbar=False, vmin=vmin, vmax=vmax
    )

    #ax.set_title(model, fontsize=13)
    ax.set_title(short_names[model], fontsize=13)
    ax.set_xlabel(r"$\varepsilon$", fontsize=12)
    ax.set_ylabel(r"$\delta / \varepsilon$", fontsize=12)

# Adjust layout to leave space for colorbar
plt.tight_layout(rect=[0, 0, 0.93, 0.95])

# Add colorbar on the right
cbar_ax = fig.add_axes([0.94, 0.25, 0.015, 0.5])  # [left, bottom, width, height]
norm = plt.Normalize(vmin=vmin, vmax=vmax)
sm = plt.cm.ScalarMappable(cmap="RdYlGn", norm=norm)
sm.set_array([])
fig.colorbar(sm, cax=cbar_ax, label='$1 - p_1$')

fig.suptitle(f"Softmax Shift Robustness (1 - $p_1$)", fontsize=16)
plt.show()


In [ ]:
import pandas as pd

# assuming df is your big dataframe
summary = (
    df_flat.groupby("model")["1-p2"]
      .agg(["mean", "std", "min", "max"])
      .reset_index()
      .round(3)
)

print(summary)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Ensure correct model order for consistency
model_order = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]
palette = sns.color_palette("Set2", n_colors=4)

# # Optional: choose one N (or loop if needed)
# N_select = 100
# df_plot = df_flat[df_flat["N"] == N_select]

plt.figure(figsize=(10, 6))
sns.violinplot(
    data=df_flat,
    x="epsilon",
    y="1-p2",
    hue="model",
    hue_order=model_order,
    palette=palette,
    cut=0,
    inner="quartile",
    linewidth=1
)

plt.title(f"Label Invariance Probability ($1 - p_2$) across models")
plt.xlabel(r"Adversarial strength $\varepsilon$")
plt.ylabel(r"$1-p_2$")
plt.ylim(0, 1)
plt.legend(title="Model", bbox_to_anchor=(0.02, 0.05))
plt.tight_layout()
plt.grid(True, axis="y")
plt.show()

In [14]:
model_names = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]
entropy_data = {}  # store outputs

for model in model_names:
    probs = relu_fits[model]["posterior"].stan_variable("prob_test")  # shape: (S, N_test, 2)

    mean_probs = np.mean(probs, axis=0)
    predictive_entropy = -np.sum(mean_probs * np.log(mean_probs + 1e-12), axis=1)

    sample_entropies = -np.sum(probs * np.log(probs + 1e-12), axis=2)
    expected_entropy = np.mean(sample_entropies, axis=0)

    mutual_information = predictive_entropy - expected_entropy

    y_pred = np.argmax(mean_probs, axis=1) + 1  # add 1 only if y_test is 1/2
    correct = (y_pred == y_test)

    # Store everything
    entropy_data[model] = {
        "entropy": predictive_entropy,
        "mutual_info": mutual_information,
        "correct": correct
    }


In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 2, figsize=(14, 12))
axs = axs.flatten()

for i, model in enumerate(model_names):
    ax = axs[i]
    entropy = entropy_data[model]["entropy"]
    mi = entropy_data[model]["mutual_info"]
    correct = entropy_data[model]["correct"]

    ax.scatter(entropy[correct], mi[correct], color='green', alpha=0.5, label='Correct')
    ax.scatter(entropy[~correct], mi[~correct], color='red', alpha=0.5, label='Incorrect')

    ax.set_title(model)
    ax.set_xlabel("Predictive Entropy")
    ax.set_ylabel("Mutual Information")
    ax.grid(True)

# Legend only once
axs[0].legend()
plt.suptitle("Mutual Information vs Entropy Colored by Prediction Correctness", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
for model in model_names:
    correct = entropy_data[model]["correct"]
    acc = np.mean(correct)
    avg_entropy = np.mean(entropy_data[model]["entropy"])
    avg_mi = np.mean(entropy_data[model]["mutual_info"])
    print(f"{model}: Accuracy={acc:.3f}, Avg Entropy={avg_entropy:.3f}, Avg MI={avg_mi:.3f}")


In [17]:
model_names = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh"]
entropy_data = {}  # store outputs

for model in model_names:
    probs = tanh_fits[model]["posterior"].stan_variable("prob_test")  # shape: (S, N_test, 2)

    mean_probs = np.mean(probs, axis=0)
    predictive_entropy = -np.sum(mean_probs * np.log(mean_probs + 1e-12), axis=1)

    sample_entropies = -np.sum(probs * np.log(probs + 1e-12), axis=2)
    expected_entropy = np.mean(sample_entropies, axis=0)

    mutual_information = predictive_entropy - expected_entropy

    y_pred = np.argmax(mean_probs, axis=1) + 1  # add 1 only if y_test is 1/2
    correct = (y_pred == y_test)

    # Store everything
    entropy_data[model] = {
        "entropy": predictive_entropy,
        "mutual_info": mutual_information,
        "correct": correct
    }


In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 2, figsize=(14, 12))
axs = axs.flatten()

for i, model in enumerate(model_names):
    ax = axs[i]
    entropy = entropy_data[model]["entropy"]
    mi = entropy_data[model]["mutual_info"]
    correct = entropy_data[model]["correct"]

    ax.scatter(entropy[correct], mi[correct], color='green', alpha=0.5, label='Correct')
    ax.scatter(entropy[~correct], mi[~correct], color='red', alpha=0.5, label='Incorrect')

    ax.set_title(model)
    ax.set_xlabel("Predictive Entropy")
    ax.set_ylabel("Mutual Information")
    ax.grid(True)

# Legend only once
axs[0].legend()
plt.suptitle("Mutual Information vs Entropy Colored by Prediction Correctness", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
for model in model_names:
    correct = entropy_data[model]["correct"]
    acc = np.mean(correct)
    avg_entropy = np.mean(entropy_data[model]["entropy"])
    avg_mi = np.mean(entropy_data[model]["mutual_info"])
    print(f"{model}: Accuracy={acc:.3f}, Avg Entropy={avg_entropy:.3f}, Avg MI={avg_mi:.3f}")


In [ ]:
import seaborn as sns
import pandas as pd

# Create a DataFrame first
df = pd.DataFrame({
    "Entropy": entropy_data["Dirichlet Student T tanh"]["entropy"],
    "MI": entropy_data["Dirichlet Student T tanh"]["mutual_info"],
    "Correct": entropy_data["Dirichlet Student T tanh"]["correct"]
})

# Plot
sns.jointplot(
    data=df,
    x="Entropy",
    y="MI",
    hue="Correct",
    kind="scatter",
    palette={True: "green", False: "red"},
    alpha=0.5,
    marginal_kws=dict(fill=True)
)
